# ResNet Ablation

## 실습목표

* 직접 ResNet 구현하기
* 모델을 config에 따라서 변경 가능하도록 만들기
* 직접 실험해서 성능 비교하기

## 학습내용

1. Ablation Study
2. CIFAR-10 데이터셋 준비
3. 블록 구성
4. VGG Complete Model
5. VGG-16 vs VGG-19
6. ResNet Ablation Study

## Ablation Study

* Ablation Study : 제거 연구
* 딥러닝에서 해당 방법을 제거한 모델과 제거하지 않은 모델의 결과를 비교하는 연구
* ResNet이 residual connection의 애블레이션 연구

## CIFAR-10 dataset 준비하기

### CIFAR-10

* Dataset 준비

In [4]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

* GPU 장착 확인

In [5]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

* tfds.load() : default로 ~/tensorflow_datasets 경로에 데이터셋을 다운로드
* data_dir : 데이터셋 경로 변경

In [ ]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir = 'data'
)

* feature 정보 확인

In [ ]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

In [ ]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

### Input Normalization

* Data Normalization : 데이터의 범위를 사용자가 원하는대로 제한하는 것
* 최댓값인 255로 정규화(normalize) 해주어 이미지 표현이 0과 1 사이로 들어오도록 해줌

In [ ]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

* label 확인

In [ ]:
ds_info.features["label"].num_classes

In [ ]:
ds_info.features["label"].names

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
fig = tfds.show_examples(ds_test, ds_info)

## 블록 구성하기

### ResNet 구조

* 블록(Block) : 주요 구조를 모듈화하여 조금씩 바꾸어 쓸 수 있는 단위
* 레이어(Layer) : Tensroflow, Keras, Pytorch 등에서 기본적으로 제공하는 단위

<img src = 'image/Architecture1.png' height = 70% width = 70%>

(from Deep Residual Learning for Image Recognition Paper)

* ResNet을 보면 ResNet-18, 34, 50, 101, 152로 다섯가지 네트워크가 있습니다.
* 일일히 따로 만들기는 불편하기에 기본 구조인 블록부터 구현하겠습니다.

### VGG 기본 구조 만들기

* CNN 레이어 여러 개와 Max Pooling 레이어 한 개로 이루어짐
* CNN 커널은 모두 3x3 라는 대표적인 특징이 있음
* 블록 내 CNN 레이어 채널의 수는 유지, 다른 블록에서 변경 될 수 있다.
* 블록 마지막에는 항상 Max Pooling 레이어가 붙음
* block_num : 레이어의 이름을 붙여주기 위해 추가
* input_shape : summary를 출력하기 위해 넣어줌

In [ ]:
# input_layer : 초기 입력값
# num_cnn : convolution layer의 갯수
# channel : 채널 수
# block_num : 블록 개수
def build_vgg_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    ):
    # 입력 레이어
    x = input_layer

    # CNN 레이어
    # filters : 채널 갯수 넣기
    # VGG에서는 커널 사이즈(filter size)가 (3,3)으로 고정입니다.
    # 활성화 함수 : relu
    # kernel_initializer='he_normal'
    # 커널 Weight Matrix의 초기값입니다. - he_normalization : 정규분포를 활용한 초기화 방법입니다.
    # padding (https://ardino.tistory.com/40)
    # valid : padding을 추가하지 않는 경우
    # full : 입력데이터의 모든 원소가 합성곱 연산에 같은 비율로 참여하도록 하는 패딩 방식을 풀 패딩(full padding)
    # same : 출력 크기를 입력크기와 동일하게 함.
    # name : 이름 설정 'block 블록 수_conv cnn수 
    for cnn_num in range(num_cnn):
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)    

    # Max Pooling 레이어
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2,
        name=f'block{block_num}_pooling'
    )(x)

    return x

* 위 함수를 통해 VGG 블록을 input_layer에 추가해봅시다.
* 입력 레이어는 데이터 셋에서 확인한 (32,32,3)
* 이 입력 레이어를 인자로 받아 build_vgg_block() 는 블록 레이어를 build 하고 출력값을 얻을 수 있습니다.
* keras의 Model 클래스에서 input과 output을 정의해 주면 간단히 블록 모델을 확인해 볼 수 있습니다.
* [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model)

In [ ]:
vgg_input_layer = keras.layers.Input(shape=(32,32,3))   # 입력 레이어 생성
vgg_block_output = build_vgg_block(vgg_input_layer)    # VGG 블록 생성

In [ ]:
# 블록 1개짜리 model 생성
model = keras.Model(inputs=vgg_input_layer, outputs=vgg_block_output)  

model.summary()

## VGG Complete Model

### VGG - 16

In [ ]:
# input_shape : 입력값의 크기
# num_cnn_list : 블록당 convolution layer 갯수의 리스트
# channel_list : 블록당 channel 수의 리스트
# num_classes : 최종 label(class)의 갯수

def build_vgg(input_shape=(32,32,3),
              num_cnn_list=[2,2,3,3,3],
              channel_list=[64,128,256,512,512],
              num_classes=10):
    
    # 블록마다 cnn과 channel을 설정해주기 때문에 
    # num_cnn_list의 내부 원소 개수와 channel_list의 내부 원소 개수가 동일해야 합니다.
    assert len(num_cnn_list) == len(channel_list)
    
    # input layer를 만들어둡니다.
    input_layer = keras.layers.Input(shape=input_shape)
    output = input_layer
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_vgg_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i
        )
    
    # Dense layer에 넣어주기 위해 flatten 합니다.
    output = keras.layers.Flatten(name='flatten')(output)
    # Dense 1차
    output = keras.layers.Dense(4096, activation='relu', name='fc1')(output)
    # Dense 2차
    output = keras.layers.Dense(4096, activation='relu', name='fc2')(output)
    # 10개의 label 값에 대한 확률을 나타내기 위해 softmax 활성화 함수를 적용합니다.
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [ ]:
vgg_16 = build_vgg()

vgg_16.summary()

### VGG - 19

In [ ]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
vgg_19 = build_vgg(
    num_cnn_list=[2,2,4,4,4],
    channel_list=[64,128,256,512,512]
)

vgg_19.summary()

## VGG-16 vs VGG-19

### CIFAR-10 불러오기

In [ ]:
BATCH_SIZE = 256
EPOCH = 15

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

* VGG 16 만들고 훈련

In [ ]:
vgg_16.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_16 = vgg_16.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

* VGG 19 만들고 훈련

In [ ]:
vgg_19.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_19 = vgg_19.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

* 훈련 손실이 어떻게 다르게 진행되는지 비교

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_16.history['loss'], 'r')
plt.plot(history_19.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()

* 검증 정확도를 비교

In [ ]:
plt.plot(history_16.history['val_accuracy'], 'r')
plt.plot(history_19.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()

## ResNet Ablation Study

### 라이브러리 확인

In [6]:
import tensorflow as tf
import numpy as np

### ResNet 기본 블럭 구성하기

### ResNet-34

In [39]:
def build_resnet_block_1(input_layer,
                       num_cnn=3, 
                       channel=64,
                       block_num=1,
                       ):
    # 입력 레이어
    x = input_layer
    isInit = 1
    
    # CNN 레이어 num_cnn만큼 반복
    # filter는 위 채널
    for cnn_num in range(num_cnn):
        
        # 처음 값을 Add layer에 넣어줍니다.
        # 단 처음 실행될때는 maxpooling을 넣습니다.
        addB = x
        
        # 처음 실행될때만 실행 한다. stage2_0_maxpooling
        # 3x3 max pool, stride 2
        if isInit == 1:
            x = keras.layers.MaxPooling2D(pool_size=(3, 3),
                                            strides = 2,
                                            padding = 'same',
                                            name = f'stage{block_num}_{cnn_num}_maxpooling'
                                            )(x)
            # Add layer에 넣을 두번째 값
            addB = x
            
        # 첫번째 CNN
        x = keras.layers.Conv2D(
            filters = channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'stage{block_num}_{cnn_num + 1}_conv1'
        )(x)
        
        # 첫번째 BatchNormalization
        x = keras.layers.BatchNormalization(
            name = f'stage{block_num}_{cnn_num + 1}_bn1'
            )(x)
        
        # 첫번째 Activation 적용
        x = keras.layers.Activation('relu')(x)
        
        # 두번째 CNN
        x = keras.layers.Conv2D(
            filters = channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'stage{block_num}_{cnn_num + 1}_conv2'
        )(x)
        
        # 두번째 BatchNormalization
        x = keras.layers.BatchNormalization(
            name = f'stage{block_num}_{cnn_num + 1}_bn2'
            )(x)
        addA = x
        
        # Add layer
        x = keras.layers.Add(
            name = f'stage{block_num}_{cnn_num + 1}_add'
            )([addA, addB])
        
        # 두번째 Activation 적용
        x = keras.layers.Activation('relu')(x)
        
        # 실행되었으므로 isInit 값을 바꿔줌
        isInit = 0
        
    return x

In [40]:
def build_resnet_block_2(input_layer,
                       num_cnn=3, 
                       channel=64,
                       block_num=1,
                       ):
    # 입력 레이어
    x = input_layer
    isInit = 1
    
    # CNN 레이어 num_cnn만큼 반복
    # filter는 위 채널
    for cnn_num in range(num_cnn):
        
        # 처음 값을 Add layer에 넣어줍니다.
        # 단 처음 실행될때는 short - batchnormalization을 넣습니다.
        addB = x
        
        # 처음 실행될때만 실행 한다
        # size를 2배 낮추는 layer - padding 방법을 변경하였다.
        if isInit == 1:
            x = keras.layers.Conv2D(
            filters = channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            strides = 2,
            padding='same',
            name=f'stage{block_num}_{cnn_num + 1}_conv1'
            )(x)
        
        # 처음 실행이 아닐 때 통상 Conv2D
        else:
            x = keras.layers.Conv2D(
                filters = channel,
                kernel_size=(3,3),
                activation='relu',
                kernel_initializer='he_normal',
                padding='same',
                name=f'stage{block_num}_{cnn_num + 1}_conv1'
            )(x)
        
        # 첫번째 BatchNormalization
        x = keras.layers.BatchNormalization(
            name = f'stage{block_num}_{cnn_num + 1}_bn1'
            )(x)
        
        # 첫번째 Activation 적용
        x = keras.layers.Activation('relu')(x)
        
        # 두번째 CNN
        x = keras.layers.Conv2D(
            filters = channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'stage{block_num}_{cnn_num + 1}_conv2'
        )(x)
        
        # 첫 번째 실행에서
        # short layer
        if isInit == 1:
            addB = keras.layers.Conv2D(
            filters = channel,
            kernel_size=(1,1),
            activation='relu',
            kernel_initializer='he_normal',
            strides = 2,
            padding='same',
            name=f'stage{block_num}_{cnn_num + 1}_short'
            )(addB)
        
        # 두번째 BatchNormalization 
        x = keras.layers.BatchNormalization(
            name = f'stage{block_num}_{cnn_num + 1}_bn2'
            )(x)
        addA = x
        
        # 첫 번째 실행에서
        # short BatchNormalization
        if isInit == 1:
            addB = keras.layers.BatchNormalization(
                name = f'stage{block_num}_{cnn_num + 1}_bn4'
                )(addB)
        
        # Add layer
        x = keras.layers.Add(
            name = f'stage{block_num}_{cnn_num + 1}_add'
            )([addA, addB])
        
        # 두번째 Activation 적용
        x = keras.layers.Activation('relu')(x)
        
        # 실행되었으므로 isInit 값을 바꿔줌
        isInit = 0
        
    return x

In [41]:
def build_resnet(input_shape=(32,32,3),
                 num_cnn_list=[3,4,6,3],
                 channel_list=[64,128,256,512],
                 num_classes=10,
                 is_50 = False
                 ):
    
    # 블록마다 cnn과 channel을 설정해주기 때문에
    # num_cnn_list의 내부 원소 개수와 channel_list의 내부 원소 개수가 동일해야 합니다.
    assert len(num_cnn_list) == len(channel_list)
    
    # input layer를 만들어둡니다.
    input_layer = keras.layers.Input(shape=input_shape, name = 'input_layer')
    output = input_layer
    
    # 첫번째 Conv2D 입니다, (None, 16, 16, 64)로 만듭니다.
    # 7x7, 64, stride 2
    output = keras.layers.Conv2D(filters=64,
                                 kernel_size=(7,7),
                                 activation='relu',
                                 kernel_initializer='he_normal',
                                 padding='same',
                                 strides = 2,
                                 )(output)
    
    # BatchNormalization, Activation 을 적용합니다.
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    # block 숫자 시작이 2이므로 i = 0 에서 시작을 i + 2로 만들어줌
    is_init_block = 1
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        
        # 첫번째 블록일때 만 1번 함수를 실행
        if is_init_block == 1:
            output = build_resnet_block_1(
                output,
                num_cnn = num_cnn, 
                channel = channel,
                block_num = i + 2
            )
            # 블록 생성함.
            is_init_block = 0
        else:
            output = build_resnet_block_2(
                output,
                num_cnn = num_cnn, 
                channel = channel,
                block_num = i + 2
            )
            
    # Dense layer에 넣어주기 위해 flatten 합니다.
    output = keras.layers.Flatten(name='flatten')(output)
    # 10개의 label 값에 대한 확률을 나타내기 위해 softmax 활성화 함수를 적용합니다.
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    
    return model

In [42]:
resnet_34 = build_resnet(input_shape=(32, 32,3), is_50=False)
resnet_34.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_11 (Conv2D)             (None, 16, 16, 64)   9472        ['input_layer[0][0]']            
                                                                                                  
 batch_normalization_11 (BatchN  (None, 16, 16, 64)  256         ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_113 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_11[0]